In [3]:
import numpy as np
from ueg import ueg_qc
from pyscf import lib, gto, scf, ao2mo, cc
from ueg import my_ueg, ueg_qc
from ad_afqmc import pyscf_interface
einsum = lib.einsum
import time

In [18]:
nocc = 7 # 7, 19, 27, 33, 57, 81, 93
system = ueg_qc(1.0, (nocc, nocc), e_cut_red=5) # this e_cut is for ueg_qc().get_k_points()
# k_points = system.get_k_points()
my_sys = my_ueg(rs=1.0, nelec=(nocc, nocc))
kpts = my_sys.get_kpts(np.sqrt(2)) # get k_pts by my_ueg, this allows k_cut = gamma*k_fermi
nkpts = kpts.shape[0]
print(f"number of k-points: {nkpts}")
h0 = system.madelung() / 2
h1 = system.get_h1_real(kpts)
eri = system.get_eri_tensor_real(kpts)
# eri = np.array(eri)

mol = gto.M()
mol.nelectron = system.n_particles
mol.nao = nkpts
mol.incore_anyway = True
mol.max_memory = 20000
# mol.energy_nuc = lambda *args: h0
mol.verbose = 4

dm = np.zeros((nkpts,nkpts))
dm[:nocc,:nocc] = np.eye(nocc) * 2.0

t0 = time.time()
mf = scf.RHF(mol)
mf.energy_nuc = lambda *args: h0
mf.get_hcore = lambda *args: h1
mf.get_ovlp = lambda *args: np.eye(nkpts)
# mf.get_jk = get_jk
mf.verbose = 4
mf._eri = ao2mo.restore(8, eri, nkpts)
mf.init_guess = "1e"
mf.max_cycle = -1
# mf.mo_coeff = np.eye(n_kpts)
emf = mf.kernel(dm0 = dm)
mf.mo_coeff = np.eye(nkpts)
t1 = time.time()

print(f"escf = {emf}, time = {t1-t0:.2f}")
print(f"escf per electron = {emf/system.n_particles}")

t0 = time.time()
mycc = cc.RCCSD(mf)
mycc.max_cycle = 100
# mycc1.conv_tol = 1e-7
eccsd = mycc.kernel()[0]
eccsd_t = mycc.ccsd_t()
t1 = time.time()

print(f"CCSD energy = {eccsd}")
print(f"CCSD(T) energy = {eccsd_t}, time = {t1-t0:.2f}")

number of k-points: 19


******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = 1e
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = -1
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /tmp/tmpa8u45h1g
max_memory 20000 MB (current use 790 MB)
Set gradient conv threshold to 3.16228e-05
init E= 8.49148146743777
  HOMO = 1.04145818327265  LUMO = 2.32324526529525
SCF not converged.
SCF energy = 8.49148146743777
escf = 8.491481467437769, time = 0.04
escf per electron = 0.6065343905312692

******** <class 'pyscf.cc.ccsd.CCSD'> ********
CC2 = 0
CCSD nocc = 7, nmo = 19
max_cycle = 100
direct = 0
conv_tol = 1e-07
conv_tol_normt = 1e-05
diis_space = 6
diis_start_cycle = 0
diis_start_energy_diff = 1e+09
max_memory 20000 MB (current use 795 MB)
Init t2, MP2 energy = 8.11699308199585  E_corr(MP2) -0.374488385441918
Ini

Overwritten attributes  energy_nuc get_hcore get_ovlp  of <class 'pyscf.scf.hf.RHF'>


In [33]:
def pw2real(self, kpts, with_zero=True):
    '''
    get the unitary transformation that
    transforms plane-wave basis to cos, sin basis.
    kpts ordered in +k, -k pairs:
    [coskx,  =  1/sqrt(2)[[ 1, 1]  [exp(ikx),
     sinkx]               [-i, i]]  exp(-ikx)]
    with_zero: add gamma point
    '''
    nkpts = kpts.shape[0]
    blk = np.array([[1.0, 1.0], [-1.0j, 1.0j]], dtype=np.complex128) / np.sqrt(2)
    
    if with_zero:
        nblks = (nkpts - 1) // 2
        u = np.kron(np.eye(nblks), blk)
        u = np.block([
            [np.array([[1.0]]), np.zeros((1, 2*nblks))],
            [np.zeros((2*nblks, 1)), u]
            ])
    elif not with_zero:
        nblks = nkpts // 2
        u = np.kron(np.eye(nblks), blk)

    return u

In [35]:
nocc = 7
my_sys = my_ueg(rs=1.0, nelec=(nocc, nocc))
kpts = my_sys.get_kpts(np.sqrt(2))
my_h1 = my_sys.get_h1(kpts)
u = pw2real(my_sys, kpts)
my_h1 = u.conj() @ my_h1 @ u.T
print(abs(my_h1-h1).max())

0.0


In [28]:
def get_mpts(self,npts):
    '''
    calculate the q = k1-k3 on integer grid points.
    m should effectively lives in a sphere of Mcut = 2*Ncut
    But since the discrete nature of lattice points, its
    safer to calculate {m} by {n} directly than using 2*Ncut. 
    #mpts ~ 4pi/3*8 #npts
    return: {m = n1-n3|n1, n3 in npts, m != 0}
    '''
    mpts = np.array([[],[],[]]).T
    for n1 in npts:
        for n3 in npts:
            mpts = np.vstack((mpts,n1-n3))

    can_mpts = self.canonical_sign(mpts)
    unq_mpts = np.unique(can_mpts, axis=0)
    unq_mpts = unq_mpts[np.argsort(np.sum(unq_mpts**2, axis=1))]
    sort_mpts = [[unq_mpts[i],-1*unq_mpts[i]] for i in range(1,len(unq_mpts))]
    sort_mpts = np.vstack(sort_mpts)

    return sort_mpts

def get_qpts(self,mpts):
    '''
    q = k1-k3
    '''
    rs = self.rs
    Np = sum(self.nelec)
    
    L = (4*np.pi*Np/3)**(1/3) * rs
    qpts = mpts * (2*np.pi/L)
    
    return qpts

def get_vq(self, qpts):
    '''
    V(q) = 4pi / q^2 / V_cell
    '''
    q2 = np.sum(qpts**2, axis=1)
    vq = 4*np.pi / q2 / self.volume
    return vq

In [37]:
def k1k3q(self, kpts, qpts):
    '''<k1,k3|q> = delta(k1-k3,q)'''
    d = np.empty((len(kpts),len(kpts),len(qpts)))
    for i,k1 in enumerate(kpts):
        for j,k3 in enumerate(kpts):
            for l,q in enumerate(qpts):
                if np.linalg.norm(k1-k3-q) < 1e-12:
                    # q2 = np.sum(q**2)
                    # vq = 4*np.pi / q2 / self.volume
                    d[i,j,l] = 1.
                else:
                    d[i,j,l] = 0.
    return d

def n1n3m(self, npts, mpts, qpts):
    '''delta(n1-n3,m)*V(q)^1/2'''
    d = np.empty((len(mpts),len(npts),len(npts)))
    for p,n1 in enumerate(npts):
        for q,n3 in enumerate(npts):
            for g,m in enumerate(mpts):
                if np.linalg.norm(n1-n3-m) < 1e-12:
                    q2 = np.sum(qpts[g]**2)
                    vq = 4*np.pi / q2 / self.volume
                    d[g,p,q] = np.sqrt(vq)
                else:
                    d[g,p,q] = 0.
    return d

In [30]:
rs = my_sys.rs
Np = sum(my_sys.nelec)
Nf = (3*Np/(8*np.pi))**(1/3)
Nc = np.sqrt(2) * Nf
npts = my_sys.get_npts(Nc)
print(npts.shape)

(19, 3)


In [31]:
mpts = get_mpts(my_sys,npts)
print(mpts.shape)
qpts = get_qpts(my_sys,mpts)
vq = get_vq(my_sys,qpts)

(92, 3)


In [36]:
uq = pw2real(my_sys, qpts, with_zero=False)
print(uq.shape)

(92, 92)


In [ ]:
def get_cderi_real(self, cderi, uk, uq):
    """Calculate the ERI tensor in real basis using the unitary transformation."""
    cderi = einsum("ip,gpq->giq", uk.conj(), cderi, optimize=True)
    cderi = einsum("giq,qj->gij", cderi, uk.T, optimize=True)
    cderi = einsum("lg,gij->lij", uq, cderi, optimize=True)
    return cderi

In [81]:
d_coeff = k1k3q(my_sys, kpts, qpts)
print(d_coeff.shape)

(19, 19, 92)


In [168]:
cderi = n1n3m(my_sys, npts, mpts, qpts)
print(cderi.shape)

(92, 19, 19)


In [125]:
eri = system.get_eri_tensor(kpts)

In [145]:
print(abs(eri-eri.transpose(1,0,2,3)).max())

0.08193030639202553


In [169]:
new_eri = einsum('gpq,gsr->pqrs',cderi,cderi)

In [170]:
print(abs(new_eri-eri).max())

1.3877787807814457e-17


In [179]:
cderi_real = get_cderi_real(system, kpts, cderi)

In [172]:
print(abs(cderi_real-cderi_real.transpose(0,2,1)).max())

0.0


In [180]:
new_eri_real = einsum('gpq,grs->pqrs',cderi_real.conj(),cderi_real)

In [174]:
eri_real = system.get_eri_tensor_real(kpts)

In [175]:
print(abs(eri_real-eri_real.transpose(1,0,2,3)).max())

0.0


In [181]:
print(abs(eri_real-new_eri_real).max())

1.3877787807814457e-17


In [183]:
new_eri_real.shape

(19, 19, 19, 19)

In [190]:
def compress_symmetric(A):
    """
    Compress symmetric matrix A (n x n) into 1D array
    storing the upper triangle (including diagonal).
    """
    n = A.shape[0]
    assert A.shape == (n, n)
    idx = np.triu_indices(n)
    return A[idx]

def compress_chol(chol):
    (nchol,nao,_) = chol.shape
    comp_chol = np.empty((nchol,nao*(nao+1)//2),dtype=np.complex128)
    for n in range(nchol):
        comp_chol[n,:] = compress_symmetric(chol[n])
    return comp_chol

In [191]:
cderi_real_s4 = compress_chol(cderi_real)

In [192]:
t0 = time.time()
dm = np.zeros((nkpts,nkpts))
dm[:nocc,:nocc] = np.eye(nocc) * 2.0
mfdf = scf.RHF(mol).density_fit()
mfdf.energy_nuc = lambda *args: h0
mfdf.get_hcore = lambda *args: h1
mfdf.get_ovlp = lambda *args: np.eye(nkpts)
mfdf.init_guess = "1e"
mfdf.max_cycle = -1
mfdf._cderi = cderi_real_s4
emfdf = mfdf.kernel(dm0=dm)
mf.mo_coeff = np.eye(nkpts)
t1 = time.time()

print(f'SCF energy = {emf}')
print(f'DF-SCF energy = {emfdf}, error = {emfdf-emf:.2e}, time = {t1-t0:.2f}')

t0 = time.time()
myccdf = cc.CCSD(mfdf)
eccsddf = myccdf.kernel()[0]
eccsddf_t = myccdf.ccsd_t()
t1 = time.time()

print(f'CCSD energy = {eccsd}')
print(f'DF-CCSD energy = {eccsddf}, error = {eccsddf-eccsd:.2e}')
print(f"CCSD(T) energy = {eccsddf_t}, error = {eccsddf_t-eccsd_t:.2e}, time = {t1-t0:.2f}")



******** <class 'pyscf.df.df_jk.DFRHF'> ********
method = DFRHF
initial guess = 1e
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = -1
direct_scf = False
chkfile to save SCF result = /tmp/tmpr5ry2hxx
max_memory 20000 MB (current use 3745 MB)
Set gradient conv threshold to 3.16228e-05


UFuncTypeError: Cannot cast ufunc 'add' output from dtype('complex128') to dtype('float64') with casting rule 'same_kind'